In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
%matplotlib inline

# 【問題1】クロスバリデーション

In [2]:
# trainデータを読み込む
df_train = pd.read_csv('application_train.csv')
df_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# 相関関係の高い説明変数「EXT_SOURCE_1」「EXT_SOURCE_2」「EXT_SOURCE_3」と目的変数の「TRAGET」を取得する
df_X = df_train[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']]
ser_y = df_train['TARGET']

In [4]:
# NULLの値を平均で埋める
df_X['EXT_SOURCE_1'] = df_X['EXT_SOURCE_1'].fillna(df_X['EXT_SOURCE_1'].mean())
df_X['EXT_SOURCE_2'] = df_X['EXT_SOURCE_2'].fillna(df_X['EXT_SOURCE_2'].mean())
df_X['EXT_SOURCE_3'] = df_X['EXT_SOURCE_3'].fillna(df_X['EXT_SOURCE_3'].mean())

/Users/chidori/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/chidori/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/chidori/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [5]:
# ndarrayに変更
X = df_X.values
y = ser_y.values

In [6]:
# # 訓練データ75%、検証データ25%として分割
# from sklearn.model_selection import train_test_split
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 0) 

In [7]:
# 標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_std = scaler.transform(X)

In [8]:
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression

## クロスバリデーション（交差検証） 

In [9]:
# クロスバリデーション
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
kf.get_n_splits(X_std)

for train_index, test_index in kf.split(X_std):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # ロジスティック回帰
    lr = LogisticRegression(random_state=0) # ロジスティック回帰のインスタンスを生成
    lr.fit(X_train, y_train)
    y_test_pred = lr.predict_proba(X_test) # 予測
    print(y_test_pred)

TRAIN: [102504 102505 102506 ... 307508 307509 307510] TEST: [     0      1      2 ... 102501 102502 102503]
[[0.57724046 0.42275954]
 [0.93165928 0.06834072]
 [0.96765366 0.03234634]
 ...
 [0.9405574  0.0594426 ]
 [0.88658204 0.11341796]
 [0.96197794 0.03802206]]
TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [102504 102505 102506 ... 205005 205006 205007]
[[0.86106322 0.13893678]
 [0.86456757 0.13543243]
 [0.98287935 0.01712065]
 ...
 [0.57974036 0.42025964]
 [0.94210264 0.05789736]
 [0.82604299 0.17395701]]
TRAIN: [     0      1      2 ... 205005 205006 205007] TEST: [205008 205009 205010 ... 307508 307509 307510]
[[0.89192238 0.10807762]
 [0.92739751 0.07260249]
 [0.90279887 0.09720113]
 ...
 [0.91267351 0.08732649]
 [0.95630688 0.04369312]
 [0.92159555 0.07840445]]


# 【問題2】グリッドサーチ

In [10]:
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression

In [11]:
from sklearn.model_selection import GridSearchCV

# ロジスティック回帰
from sklearn.linear_model import LogisticRegression
lr2 = LogisticRegression() # ロジスティック回帰のインスタンスを生成

# パラメータ設定
parameters = {'solver':('lbfgs', 'liblinear', 'sag', 'saga'), 'C':[1, 0.8, 0.6]}

# グリッドサーチ
clf = GridSearchCV(lr2, parameters)
clf.fit(X_std, y)
sorted(clf.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_C',
 'param_solver',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

In [12]:
best_param = clf.best_params_
print(best_param)

{'C': 1, 'solver': 'lbfgs'}


In [13]:
best_estimator = clf.best_estimator_
print(best_estimator)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)


In [14]:
clf.best_score_

0.9191508596440453

# 【問題3】Kaggle Notebooksからの調査

## KaggleのNotebooksから見つけたアイデア
### クロスバリデーションには以下の複数の方法がある
### ・GroupKFold  
### ・GroupShuffleSplit  
### ・KFold  
### ・LeaveOneGroupOut  
### ・LeavePGroupsOut  
### ・LeaveOneOut  
### ・LeavePOut  
### ・PredefinedSplit  
### ・RepeatedKFold  
### ・RepeatedStratifiedKFold  
### ・ShuffleSplit  
### ・StratifiedKFold  
### ・StratifiedShuffleSplit  
### ・TimeSeriesSplit  
「StratifiedKFold」を使用するとラベルの比率が揃うようにtrainデータとtestデータを分けてくれる  
今回のデータセットに利用できそうな「StratifiedKFold」を用いて高い汎化性能を出す

# 【問題4】高い汎化性能のモデル作成

## クロスバリデーション「StratifiedKFold」で分割数を変更して汎化性能を比較する

In [15]:
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression

# データ検証用
from sklearn.metrics import roc_auc_score

In [16]:
# クロスバリデーション（StratifiedKFold）
from sklearn.model_selection import StratifiedKFold

def check_skf(X, y, splits):
    
    skf = StratifiedKFold(n_splits=splits)
    skf.get_n_splits(X, y)

    lst_skf_auc = []

    for train_index, test_index in skf.split(X, y):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # ロジスティック回帰
        lr = LogisticRegression() # ロジスティック回帰のインスタンスを生成
        lr.fit(X_train, y_train)
        y_test_pred = lr.predict_proba(X_test) # 予測
        lst_skf_auc.append(roc_auc_score(y_test,y_test_pred[:,1]))
    
    ser_skf_auc = pd.Series(lst_skf_auc)
    return ser_skf_auc.mean()

In [17]:
for split in range(3, 20, 1):
    skf_auc_mean = check_skf(X_std, y, split)
    print("分割数：{} 予測値の平均AUC：{}".format(split, skf_auc_mean))

分割数：3 予測値の平均AUC：0.7189488556676107
分割数：4 予測値の平均AUC：0.71896268277955
分割数：5 予測値の平均AUC：0.718945033507176
分割数：6 予測値の平均AUC：0.7189474910445511
分割数：7 予測値の平均AUC：0.7189458926094956
分割数：8 予測値の平均AUC：0.7189446298682297
分割数：9 予測値の平均AUC：0.7189408001292431
分割数：10 予測値の平均AUC：0.7189353656354219
分割数：11 予測値の平均AUC：0.7189329341661217
分割数：12 予測値の平均AUC：0.7189333190613895
分割数：13 予測値の平均AUC：0.718940341891913
分割数：14 予測値の平均AUC：0.7189484301896983
分割数：15 予測値の平均AUC：0.7189600697784039
分割数：16 予測値の平均AUC：0.7189280060304656
分割数：17 予測値の平均AUC：0.718929955557136
分割数：18 予測値の平均AUC：0.7189296348336593
分割数：19 予測値の平均AUC：0.7189392803450138


## 結果
分割数が「4」のとき予測値の平均が最も良くなった。

# 【問題5】最終的なモデルの選定

## 「ロジスティック回帰」でグリッドサーチを用いて汎化性能の高いモデルを作成する

In [18]:
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression
lr3 = LogisticRegression() # ロジスティック回帰のインスタンスを生成

# パラメータ設定
parameters = {'solver':('lbfgs', 'liblinear', 'sag', 'saga'), 'C':[0.6, 0.8, 1, 2, 3]}

# グリッドサーチ
clf = GridSearchCV(lr3, parameters)
clf.fit(X_std, y)
sorted(clf.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_C',
 'param_solver',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

In [19]:
best_param = clf.best_params_
print(best_param)

{'C': 0.6, 'solver': 'lbfgs'}


In [20]:
clf.best_estimator_

LogisticRegression(C=0.6, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)

## 上記の結果をもとに「ロジスティック回帰」で予測する

In [21]:
# testデータを読み込む
df_test = pd.read_csv('application_test.csv')
df_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# 説明変数「EXT_SOURCE_1」「EXT_SOURCE_2」「EXT_SOURCE_3」を取得する
X_test = df_test[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']]

In [23]:
# NULLの値を平均で埋める
X_test['EXT_SOURCE_1'] = X_test['EXT_SOURCE_1'].fillna(X_test['EXT_SOURCE_1'].mean())
X_test['EXT_SOURCE_2'] = X_test['EXT_SOURCE_2'].fillna(X_test['EXT_SOURCE_2'].mean())
X_test['EXT_SOURCE_3'] = X_test['EXT_SOURCE_3'].fillna(X_test['EXT_SOURCE_3'].mean())

/Users/chidori/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/chidori/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/chidori/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [24]:
# ndarrayに変換
X_test = X_test.values

In [25]:
# 標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_test)
X_test_std = scaler.transform(X_test)

In [26]:
lr4 = LogisticRegression(C=0.8, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)
clf.fit(X_std, y)
X_test_pred = clf.predict_proba(X_test_std)

In [27]:
Xy_test_pred = pd.concat([df_test['SK_ID_CURR'], pd.Series(y_test_pred[:,1])], axis = 1)
Xy_test_pred.columns = ['SK_ID_CURR','TARGET']
Xy_test_pred.head()

,SK_ID_CURR,TARGET
0,100001.0,0.108078
1,100005.0,0.072602
2,100013.0,0.097201
3,100028.0,0.027510
4,100038.0,0.087779


In [28]:
# CSVファイルに出力してKaggleに提出
Xy_test_pred.to_csv('pred.csv', header=True, index=False)